<a href="https://colab.research.google.com/github/umangjpatel/g20-machine-learning/blob/main/Siapkmed%20Stratified%205%20fold%20cross-validation%20demo%20using%20Fastai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastai.vision.all import *
!pip install kaggle
!pip install timm
import timm
from google.colab import files
from pathlib import Path
kaggle_key = (Path(".")) / "kaggle.json"
from sklearn.model_selection import StratifiedKFold
from fastai.callback.tracker import SaveModelCallback
if not kaggle_key.exists():
  uploaded = files.upload()
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets download umangjpatel/sipakmed-wsi -p="dataset" --unzip

     |████████████████████████████████| 727kB 17.3MB/s 
     |████████████████████████████████| 194kB 56.1MB/s 
     |████████████████████████████████| 51kB 8.4MB/s 
     |████████████████████████████████| 1.2MB 54.6MB/s 
     |████████████████████████████████| 776.8MB 22kB/s 
     |████████████████████████████████| 12.8MB 249kB/s 
     |████████████████████████████████| 61kB 8.8MB/s 
     |████████████████████████████████| 51kB 6.7MB/s 
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.7.1 which is incompatible.
Mounted at /content/gdrive
     |████████████████████████████████| 296kB 18.9MB/s 


Saving kaggle.json to kaggle.json
100% 6.00G/6.01G [02:10<00:00, 45.4MB/s]
100% 6.01G/6.01G [02:10<00:00, 49.3MB/s]


In [2]:
current_path = Path(".")
dataset_path = current_path / "dataset"
dataset_path.ls()

(#5) [Path('dataset/koilocytotic'),Path('dataset/superficial_intermediate'),Path('dataset/parabasal'),Path('dataset/metaplastic'),Path('dataset/dyskeratotic')]

In [3]:
strat_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)

In [4]:
labels = L([img.parent.name for img in get_image_files(dataset_path)])
labels

(#966) ['koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic','koilocytotic'...]

In [5]:
for i, (fold_train, fold_valid) in enumerate(strat_fold.split(X=get_image_files(dataset_path), y=labels)):
  print(f"Fold #{i+1} => Train : {len(fold_train)}, valid : {len(fold_valid)}")

Fold #1 => Train : 772, valid : 194
Fold #2 => Train : 773, valid : 193
Fold #3 => Train : 773, valid : 193
Fold #4 => Train : 773, valid : 193
Fold #5 => Train : 773, valid : 193


In [6]:
k_fold_idxs = L(list(strat_fold.split(X=get_image_files(dataset_path), y=labels)))

In [7]:
def get_images_by_idx(fold_idxs):
  idxs = list(fold_idxs[0]) + list(fold_idxs[1])
  imgs = get_image_files(dataset_path)
  return lambda _: L([imgs[idx] for idx in idxs])

def get_fold_data(fold_idxs, size, bs=64, seed=21):
  return DataBlock(
      blocks=(ImageBlock(), CategoryBlock()),
      get_items=get_images_by_idx(fold_idxs),
      get_y=parent_label,
      splitter=IndexSplitter(fold_idxs[1]),
      item_tfms=Resize(size * 2),
      batch_tfms=[*aug_transforms(max_rotate=60.0, max_zoom=1.0, flip_vert=True, size=size),
                Normalize.from_stats(*imagenet_stats)]).dataloaders(dataset_path, bs=bs, seed=seed)

for i, k_fold in enumerate(k_fold_idxs):
  data = get_fold_data(k_fold, size=224)
  print(f"Fold #{i+1} => Train : {len(data.train.items)}, valid : {len(data.valid.items)}")


Fold #1 => Train : 772, valid : 194
Fold #2 => Train : 773, valid : 193
Fold #3 => Train : 773, valid : 193
Fold #4 => Train : 773, valid : 193
Fold #5 => Train : 773, valid : 193


In [8]:
def get_metrics():
  return [accuracy, Precision(average="macro"), Recall(average="macro"),
          F1Score(average="macro"), CohenKappa(weights="quadratic")]

def save_callback(name):
  return [SaveModelCallback(monitor="accuracy", fname=name, with_opt=True)]

In [11]:
fold_1_data = get_fold_data(k_fold_idxs[0], size=224, bs=32)

In [12]:
learner = cnn_learner(fold_1_data, resnet34, metrics=get_metrics(), model_dir=current_path)
learner.fine_tune(2, cbs=save_callback("rn34-demo"))

epoch,train_loss,valid_loss,accuracy,precision_score,recall_score,f1_score,cohen_kappa_score,time
0,2.017743,0.789837,0.778351,0.776767,0.813128,0.790003,0.794791,03:41


Better model found at epoch 0 with accuracy value: 0.7783505320549011.


epoch,train_loss,valid_loss,accuracy,precision_score,recall_score,f1_score,cohen_kappa_score,time
0,0.902490,0.657665,0.798969,0.789203,0.835838,0.801135,0.741006,03:34
1,0.742281,0.463444,0.855670,0.849162,0.883962,0.863088,0.849804,03:19


Better model found at epoch 0 with accuracy value: 0.7989690899848938.
Better model found at epoch 1 with accuracy value: 0.8556700944900513.
